In [1]:
import pandas as pd
import csv
import os
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
import joblib

from sklearn.decomposition import PCA
import statsmodels.tools.tools as sm
from sklearn.metrics import confusion_matrix
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestClassifier
# from config import user_name,password

In [2]:
btc = pd.read_csv("data/BTC-USD.csv")
btc = btc.drop(columns=["Adj Close","Open","High","Low","Volume"])
btc['Date']= pd.to_datetime(btc['Date'])
btc = btc.dropna()
btc_data = btc.iloc[::-1]
# btc_data.head()

In [3]:
btc_data = btc_data.rename(columns={'Close': 'Close_Btc'})
btc_data['Date'] = pd.to_datetime(btc_data['Date'])
btc_data['Day_Btc'] = btc_data['Date'].dt.day_name()
btc_data = btc_data.sort_values(by="Date")
# btc_data

In [4]:
btc_data_clean = btc_data.loc[((btc_data['Day_Btc'] != "Saturday") & (btc_data['Day_Btc'] != 'Sunday'))]
# btc_data_clean

In [5]:
btc_data_clean = btc_data_clean.drop(columns=["Day_Btc"])
btc_data_clean["BTC_Change%"] = btc_data_clean["Close_Btc"].pct_change()
btc_data_clean['BTC_Change%_lag'] = btc_data_clean["BTC_Change%"].shift(1)

# Construction direction of BITCOIN movement
btc_data_clean['BTC_dir'] = (btc_data_clean["BTC_Change%"]>0).astype(int)
btc_data_clean['BTC_dir_lag'] = btc_data_clean['BTC_dir'].shift(1)
# btc_data_clean

In [6]:
btc_data_clean.dtypes

Date               datetime64[ns]
Close_Btc                 float64
BTC_Change%               float64
BTC_Change%_lag           float64
BTC_dir                     int64
BTC_dir_lag               float64
dtype: object

In [7]:
eth = pd.read_csv("data/ETH-USD.csv")
eth = eth.drop(columns=["Adj Close","Open","High","Low","Volume"])
eth['Date']= pd.to_datetime(eth['Date'])
eth_data = eth.dropna()
eth_data = eth_data.iloc[::-1]
# eth_data.head()

In [8]:
eth_data = eth_data.rename(columns={'Close': 'Close_Eth'})
eth_data['Date'] = pd.to_datetime(eth_data['Date'])
eth_data['Day_Eth'] = eth_data['Date'].dt.day_name()
eth_data = eth_data.sort_values(by="Date")
# eth_data

In [9]:
eth_data_clean = eth_data.loc[((eth_data['Day_Eth'] != "Saturday") & (eth_data['Day_Eth'] != 'Sunday'))]
# eth_data_clean

In [10]:
eth_data_clean = eth_data_clean.drop(columns=["Day_Eth"])
eth_data_clean["ETH_Change%"] = eth_data_clean["Close_Eth"].pct_change()
eth_data_clean['ETH_Change%_lag'] = eth_data_clean["ETH_Change%"].shift(1)

# Construction direction of Ethereum movement
eth_data_clean['ETH_dir'] = (eth_data_clean["ETH_Change%"]>0).astype(float)
eth_data_clean['ETH_dir_lag'] = eth_data_clean['ETH_dir'].shift(1)
eth_data_clean.head()

,Date,Close_Eth,ETH_Change%,ETH_Change%_lag,ETH_dir,ETH_dir_lag
2,2016-10-10,11.7555,NaN,NaN,0.0,NaN
3,2016-10-11,11.7857,0.002569,NaN,1.0,0.0
4,2016-10-12,11.9335,0.012541,0.002569,1.0,1.0
5,2016-10-13,11.9630,0.002472,0.012541,1.0,1.0
6,2016-10-14,11.9382,-0.002073,0.002472,0.0,1.0


In [11]:
eth_data_clean.dtypes

Date               datetime64[ns]
Close_Eth                 float64
ETH_Change%               float64
ETH_Change%_lag           float64
ETH_dir                   float64
ETH_dir_lag               float64
dtype: object

In [12]:
gold = pd.read_csv("data/Gold-Futures.csv")
gold = gold.drop(columns=["Vol.","Change %","Open","High","Low"])
gold['Date']= pd.to_datetime(gold['Date'])
gold_data = gold.dropna()
# gold_data.head()

In [13]:
gold_data = gold_data.rename(columns={'Price': 'Price_Gold'})
gold_data['Date'] = pd.to_datetime(gold_data['Date'])
gold_data['Day_Gold'] = gold_data['Date'].dt.day_name()
gold_data = gold_data.sort_values(by="Date")
gold_data = gold_data.reset_index(drop=True)
gold_data['Price_Gold'] = gold_data.Price_Gold.str.replace(',', '').astype(float)
# gold_data

In [14]:
gold_data_clean = gold_data.loc[((gold_data['Day_Gold'] != "Saturday") & (gold_data['Day_Gold'] != 'Sunday'))]
# gold_data_clean

In [15]:
gold_data_clean = gold_data_clean.drop(columns=["Day_Gold"])
gold_data_clean["Gold_Change%"] = gold_data_clean["Price_Gold"].pct_change()
gold_data_clean['Gold_Change%_lag'] = gold_data_clean["Gold_Change%"].shift(1)

# Construction direction of GOLD movement
gold_data_clean['gold_dir'] = (gold_data_clean["Gold_Change%"]>0).astype(float)
gold_data_clean['gold_dir_lag'] = gold_data_clean['gold_dir'].shift(1)
gold_data_clean.head()

,Date,Price_Gold,Gold_Change%,Gold_Change%_lag,gold_dir,gold_dir_lag
0,2016-09-08,1400.2,NaN,NaN,0.0,NaN
1,2016-09-09,1393.1,-0.005071,NaN,0.0,0.0
2,2016-09-12,1384.4,-0.006245,-0.005071,0.0,0.0
3,2016-09-13,1380.8,-0.002600,-0.006245,0.0,0.0
4,2016-09-14,1383.3,0.001811,-0.002600,1.0,0.0


In [16]:
gold_data_clean.dtypes

Date                datetime64[ns]
Price_Gold                 float64
Gold_Change%               float64
Gold_Change%_lag           float64
gold_dir                   float64
gold_dir_lag               float64
dtype: object

In [17]:
snp = pd.read_csv("data/S&P-500.csv")
snp = snp.drop(columns=["Vol.","Change %","Open","High","Low"])
snp['Date']= pd.to_datetime(snp['Date'])
snp_data = snp.dropna()
# snp_data.head()

In [18]:
snp_data = snp_data.rename(columns={'Price': 'Price_Snp'})
snp_data['Date'] = pd.to_datetime(snp_data['Date'])
snp_data['Day_Snp'] = snp_data['Date'].dt.day_name()
snp_data = snp_data.sort_values(by="Date")
snp_data = snp_data.reset_index(drop=True)
snp_data['Price_Snp'] = snp_data.Price_Snp.str.replace(',', '').astype(float)
# snp_data

In [19]:
snp_data_clean = snp_data.loc[((snp_data['Day_Snp'] != "Saturday") & (snp_data['Day_Snp'] != 'Sunday'))]
# snp_data_clean

In [20]:
snp_data_clean = snp_data_clean.drop(columns=["Day_Snp"])
snp_data_clean["Snp_Change%"] = snp_data_clean["Price_Snp"].pct_change()
snp_data_clean['Snp_Change%_lag'] = snp_data_clean["Snp_Change%"].shift(1)

# Construction direction of S&P movement
snp_data_clean['Snp_dir'] = (snp_data_clean["Snp_Change%"]>0).astype(float)
snp_data_clean['Snp_dir_lag'] = snp_data_clean['Snp_dir'].shift(1)
# snp_data_clean

In [21]:
usbond = pd.read_csv("data/US-10-Year-Bond.csv")
usbond = usbond.drop(columns=["Change %","Open","High","Low"])
usbond['Date']= pd.to_datetime(usbond['Date'])
usbond_data = usbond.dropna()
# usbond_data.head()

In [22]:
usbond_data = usbond_data.rename(columns={'Price': 'Price_USB'})
usbond_data['Date'] = pd.to_datetime(usbond_data['Date'])
usbond_data['Day_USB'] = usbond_data['Date'].dt.day_name()
usbond_data = usbond_data.sort_values(by="Date")
usbond_data = usbond_data.reset_index(drop=True)
# usbond_data

In [23]:
usbond_data_clean = usbond_data.loc[((usbond_data['Day_USB'] != "Saturday") & (usbond_data['Day_USB'] != 'Sunday'))]
# usbond_data_clean

In [24]:
usbond_data_clean = usbond_data_clean.drop(columns=["Day_USB"])
usbond_data_clean["USB_Change%"] = usbond_data_clean["Price_USB"].pct_change()
usbond_data_clean['USB_Change%_lag'] = usbond_data_clean["USB_Change%"].shift(1)

# Construction direction of US BOND movement
usbond_data_clean['USB_dir'] = (usbond_data_clean["USB_Change%"]>0).astype(float)
usbond_data_clean['USB_dir_lag'] = usbond_data_clean['USB_dir'].shift(1)
# usbond_data_clean

In [25]:
usbond_data_clean.dtypes

Date               datetime64[ns]
Price_USB                 float64
USB_Change%               float64
USB_Change%_lag           float64
USB_dir                   float64
USB_dir_lag               float64
dtype: object

In [26]:
bit_1 = btc_data_clean.merge(eth_data_clean, how='left', on='Date')
# bit_1

In [27]:
bit_2 = bit_1.merge(snp_data_clean, how='left', on='Date')
# bit_2

In [28]:
bit_3 = bit_2.merge(usbond_data_clean, how='left', on='Date')
bit_3 = bit_3.dropna()
bit_3 = bit_3.reset_index(drop=True)
bit_3.head()

,Date,Close_Btc,BTC_Change%,BTC_Change%_lag,BTC_dir,BTC_dir_lag,Close_Eth,ETH_Change%,ETH_Change%_lag,ETH_dir,...,Price_Snp,Snp_Change%,Snp_Change%_lag,Snp_dir,Snp_dir_lag,Price_USB,USB_Change%,USB_Change%_lag,USB_dir,USB_dir_lag
0,2016-10-12,636.192017,-0.007612,0.035668,0,1.0,11.9335,0.012541,0.002569,1.0,...,2139.18,0.001147,-0.012447,1.0,0.0,1.771,0.002264,0.001133,1.0,1.0
1,2016-10-13,636.786011,0.000934,-0.007612,1,0.0,11.9630,0.002472,0.012541,1.0,...,2132.55,-0.003099,0.001147,0.0,1.0,1.746,-0.014116,0.002264,0.0,1.0
2,2016-10-14,640.377991,0.005641,0.000934,1,1.0,11.9382,-0.002073,0.002472,0.0,...,2132.98,0.000202,-0.003099,1.0,0.0,1.805,0.033792,-0.014116,1.0,0.0
3,2016-10-17,639.192993,-0.001850,0.005641,0,1.0,12.0050,0.005595,-0.002073,1.0,...,2126.50,-0.003038,0.000202,0.0,1.0,1.768,-0.020499,0.033792,0.0,1.0
4,2016-10-18,637.960022,-0.001929,-0.001850,0,0.0,12.5885,0.048605,0.005595,1.0,...,2139.60,0.006160,-0.003038,1.0,0.0,1.740,-0.015837,-0.020499,0.0,0.0


In [29]:
gold_1 = gold_data_clean.merge(usbond_data_clean, how='left', on='Date')
# gold_1

In [30]:
gold_2 = gold_1.merge(snp_data_clean, how='left', on='Date')
# gold_2

In [31]:
gold_3 = gold_2.merge(eth_data_clean, how='left', on='Date')
gold_3 = gold_3.dropna()
gold_3 = gold_3.reset_index(drop=True)
# gold_3.head()

In [32]:
bit_3 = bit_3[['Date', 'BTC_dir', 'BTC_dir_lag', 'BTC_Change%_lag', 'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
bit_3.head()

,Date,BTC_dir,BTC_dir_lag,BTC_Change%_lag,USB_Change%_lag,Snp_Change%_lag,ETH_Change%_lag
0,2016-10-12,0,1.0,0.035668,0.001133,-0.012447,0.002569
1,2016-10-13,1,0.0,-0.007612,0.002264,0.001147,0.012541
2,2016-10-14,1,1.0,0.000934,-0.014116,-0.003099,0.002472
3,2016-10-17,0,1.0,0.005641,0.033792,0.000202,-0.002073
4,2016-10-18,0,0.0,-0.001850,-0.020499,-0.003038,0.005595


In [33]:
# gold_plot = gold_3[["Date","gold_dir"]]
# gold_plot.to_csv("gold_plot.csv",index=False)

In [34]:
# Logistic Model specification
logit = LogisticRegression()                                                 

days_logit = 70
    # Note: 54.55% accuracy  [[22 41] [19 50]]

# Making predictions for only latest 132 days
buffer = 132

# for days_logit in range(20, 910, 10):
pred_logit = pd.DataFrame(columns=['Date', 'BTC_dir','logit']) # DataFrame to store logit outcome

data_log = bit_3.iloc[-days_logit-buffer:]  

for j in range(0, buffer):

    train = data_log.iloc[j:j+days_logit,:] # Training Model    
    test = data_log.iloc[j+days_logit : j+days_logit+1,:] # Testing Model

    # Separating X and Y variables for easy computation
    x_train = train.drop(columns=['Date', 'BTC_dir'])        
    x_test = test.drop(columns=['Date', 'BTC_dir'])

    y_train = train['BTC_dir']
    y_test = test['BTC_dir']

    selector = RFECV(estimator=logit, cv=5, step=1)
    selector.fit(x_train, y_train)
    features_bit = x_train.columns.tolist()
    selected_features_bit = sorted(zip(selector.ranking_, features_bit))
    features_rank_bit = pd.DataFrame(selected_features_bit, columns=['Ranking', 'Feature'])
    features_rank_bit = features_rank_bit.set_index('Feature')
#         print(features_rank_bit)

    ranked_features_bit = []
    for s in selected_features_bit:
        if s[0] < 3:
            ranked_features_bit.append(s[1])

    x_train_bit_sel = x_train[ranked_features_bit]
    x_test_bit_sel = x_test[ranked_features_bit]

    logit_model = logit.fit(x_train_bit_sel, y_train) 
    y_pred_l = logit_model.predict(x_test_bit_sel)

    df_reg = pd.DataFrame({'logit':y_pred_l.tolist()})
    true_data = test[['Date', 'BTC_dir']]
    true_data = true_data.reset_index().drop(columns=['index'])
    df = pd.concat([true_data, df_reg], axis=1)
    pred_logit = pred_logit.append(df)  #Appending predictions to DF

for i in range(1, pred_logit.shape[1]):
    pred_logit.iloc[:,i] = pred_logit.iloc[:,i].astype(float);

a_logit = confusion_matrix(pred_logit['BTC_dir'], pred_logit['logit'])
b_logit = round((a_logit[0][0] + a_logit[1][1])/(a_logit[0][0]+ a_logit[1][1] + a_logit[0][1] + a_logit[1][0]), 4)
print('day_logit:', days_logit, '\n', a_logit, b_logit)

/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not h

/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not h

/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not h

/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not h

day_logit: 70 
 [[22 41]
 [19 50]] 0.5455


/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"
/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RFECV was fitted with feature names
  "X does not have valid feature names, but"


In [35]:
pred_logit.shape

(132, 3)

### XG Boost

In [36]:
days_xg = 90
    # Note: 60.61% accuracy  [[35 28] [24 45]]

# for days_xg in range(10, 910,10):
pred_xgb_r = pd.DataFrame(columns=['Date', 'BTC_dir', 'XG_bit'])

data_xg = bit_3.iloc[-days_xg-buffer:, :]

for i in range(0, buffer):
    train = data_xg.iloc[i:days_xg+i, :] # Training model
    test = data_xg.iloc[days_xg+i : days_xg+i+1, :]
#     print(train.tail(2), test)

    x_train = train.drop(columns=['BTC_dir', 'Date'])
    y_train = train['BTC_dir']

    x_test = test.drop(columns=['BTC_dir', 'Date'])
    y_test = test['BTC_dir']

    xg_mr = XGBRegressor()
    xg_m2 = xg_mr.fit(x_train, y_train)
    y_pred_xg = xg_m2.predict(x_test)

    df_reg = pd.DataFrame({'XG_bit': y_pred_xg.tolist()})
    true_data = test[['Date', 'BTC_dir']]
    true_data = true_data.reset_index(drop=True)

    df = pd.concat([true_data, df_reg], axis=1)
    pred_xgb_r = pred_xgb_r.append(df)    

#     break

for i in range(1, pred_xgb_r.shape[1]):
    pred_xgb_r.iloc[:,i] = pred_xgb_r.iloc[:,i].astype(float);
    
    
pred_xgb_r['xg_bit_r'] = (pred_xgb_r['XG_bit']>0.5).astype(float)
a_xg = confusion_matrix(pred_xgb_r['BTC_dir'], pred_xgb_r['xg_bit_r'])
print(days_xg, "\n", a_xg)
print(round ((a_xg[0][0] + a_xg[1][1]) / (a_xg[0][0]+ a_xg[1][1] + a_xg[0][1] + a_xg[1][0]), 4))

90 
 [[35 28]
 [24 45]]
0.6061


In [37]:
# XGB Classifier

days_xg = 80

# for days_xg in range(10, 910,10):
pred_xgb_c = pd.DataFrame(columns=['Date', 'BTC_dir', 'xg_bit_c'])

data_xg = bit_3.iloc[-days_xg-buffer:, :]

for i in range(0, buffer):
    train = data_xg.iloc[i:days_xg+i, :] # Training model
    test = data_xg.iloc[days_xg+i : days_xg+i+1, :]
#     print(train.tail(2), test)

    x_train = train.drop(columns=['BTC_dir', 'Date'])
    y_train = train['BTC_dir']

    x_test = test.drop(columns=['BTC_dir', 'Date'])
    y_test = test['BTC_dir']

    xg_m = XGBClassifier()
    xg_m2 = xg_m.fit(x_train, y_train)
    y_pred_xg = xg_m2.predict(x_test)

    df_reg = pd.DataFrame({'xg_bit_c': y_pred_xg.tolist()})
    true_data = test[['Date', 'BTC_dir']]
    true_data = true_data.reset_index(drop=True)

    df = pd.concat([true_data, df_reg], axis=1)
    pred_xgb_c = pred_xgb_c.append(df)    

#     break
for i in range(1, pred_xgb_c.shape[1]):
    pred_xgb_c.iloc[:,i] = pred_xgb_c.iloc[:,i].astype(float);

a_xg = confusion_matrix(pred_xgb_c['BTC_dir'], pred_xgb_c['xg_bit_c'])
print(days_xg, "\n", a_xg)
print(round ((a_xg[0][0] + a_xg[1][1]) / (a_xg[0][0]+ a_xg[1][1] + a_xg[0][1] + a_xg[1][0]), 4))

/Users/jsb/opt/anaconda3/envs/inv_stg/lib/python3.7/site-packages/xgboost/sklearn.py:1220: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


80 
 [[39 24]
 [27 42]]
0.6136


# Random Forest

In [38]:
days_rf = 370
rs_start = 4
rs_end = 5

# for days_rf in range(10,910, 10):
pred_rf = pd.DataFrame(columns=['Date', 'BTC_dir', 'rs', 'rf'])
data_rf = bit_3.iloc[-days_rf-buffer:, :]

for rs in range(rs_start, rs_end):   # Various random states
    for i in range(0, buffer):
        train = data_rf.iloc[i:days_rf+i, :] # Training model
        test = data_rf.iloc[days_rf+i : days_rf+i+1, :]
#             print(train.tail(2), test)

        # Separating X and Y variables for easy computation
        X_train = train.drop(columns=['Date', 'BTC_dir'])
        X_test = test.drop(columns=['Date', 'BTC_dir'])

        y_train = train['BTC_dir']
        y_test = test['BTC_dir']

        pca = PCA(n_components = 0.95).fit(X_train) # Principal Component Analyses

        train_pca = pca.transform(X_train)
        test_pca = pca.transform(X_test)

        # Random Forest Model
        random_forest = RandomForestClassifier(oob_score= False, n_estimators= 50, min_samples_split= 2, max_features= 'log2',
                                criterion= 'gini', bootstrap= True, random_state=rs)
#             random_forest = RandomForestClassifier()
        rf_model = random_forest.fit(train_pca, y_train) 
        y_pred_rf = rf_model.predict(test_pca)

        ls_rs = [rs]*len(y_test)

        df_reg = pd.DataFrame({'rs': ls_rs, 'rf': y_pred_rf.tolist()})
        true_data = test[['Date', 'BTC_dir']]
        true_data = true_data.reset_index().drop(columns=['index'])
        df = pd.concat([true_data, df_reg], axis=1)
        pred_rf = pred_rf.append(df)
#             print(pred_rf.tail(2))

    # Retaining only that random state with highest accuracy      
    for i in range(1, pred_rf.shape[1]): # The outputs may come in str type  
        pred_rf.iloc[:,i] = pred_rf.iloc[:,i].astype(float)

    # Estimating Confusion matrix for each model and random state
# cm = pd.DataFrame(columns=['rs', 'RF_CM', 'RF_Accuracy'])

# for rs in range(rs_start, rs_end):
#     df = pred_rf[pred_rf['rs']==rs]
#     a_rf = (confusion_matrix(df['BTC_dir'], df['rf']))
#     b_rf = round((a_rf[0][0] + a_rf[1][1])/(a_rf[0][0]+ a_rf[1][1] + a_rf[0][1] + a_rf[1][0]),4) 
#     conf_mat = pd.DataFrame({'rs': [rs], 'RF_CM': [a_rf], 'RF_Accuracy': [b_rf]}) # Appending to dataframes element-wise
#     cm = cm.append(conf_mat)


# max_acc_rs = cm['rs'][cm['RF_Accuracy']==cm['RF_Accuracy'].max()] # Retaining obs with highest accuracy
# pred_rf_max_acc = pred_rf[pred_rf['rs']==max_acc_rs.iloc[0]]
# print('days_rf:', days_rf, cm[cm['rs']==max_acc_rs.iloc[0]])

pred_rf_max_acc = pred_rf
a_rf = (confusion_matrix(pred_rf_max_acc['BTC_dir'], pred_rf_max_acc['rf']))
b_rf = round((a_rf[0][0] + a_rf[1][1])/(a_rf[0][0]+ a_rf[1][1] + a_rf[0][1] + a_rf[1][0]),4) 
a_rf, b_rf

(array([[33, 30],
        [25, 44]]),
 0.5833)

In [39]:
pred_rf_max_acc.head()

,Date,BTC_dir,rs,rf
0,2021-03-31,1.0,4.0,1.0
0,2021-04-01,1.0,4.0,0.0
0,2021-04-05,0.0,4.0,0.0
0,2021-04-06,0.0,4.0,1.0
0,2021-04-07,0.0,4.0,1.0


In [40]:
trading_str_b = pred_logit.merge(pred_xgb_r, on=['Date', 'BTC_dir'], how='inner')
trading_str_b = trading_str_b.merge(pred_xgb_c, on=['Date', 'BTC_dir'], how='inner')
trading_str_b = trading_str_b.merge(pred_rf_max_acc, on=['Date', 'BTC_dir'], how='inner')
trading_str_b = trading_str_b.drop(columns=['XG_bit', 'rs'])
trading_str_b.head()

,Date,BTC_dir,logit,xg_bit_r,xg_bit_c,rf
0,2021-03-31,1.0,1.0,0.0,0.0,1.0
1,2021-04-01,1.0,1.0,1.0,0.0,0.0
2,2021-04-05,0.0,1.0,0.0,0.0,0.0
3,2021-04-06,0.0,1.0,1.0,1.0,1.0
4,2021-04-07,0.0,1.0,0.0,0.0,1.0


In [41]:
# Ensemble output:

trading_str_b['sum'] = trading_str_b['xg_bit_r'] + trading_str_b['xg_bit_c'] + trading_str_b['rf'] + trading_str_b['logit'] 
trading_str_b['trade'] = np.where(trading_str_b['sum']==4, 1, np.where((trading_str_b['sum']==0), -1, 0))

# Determining Accuracy
a = trading_str_b['BTC_dir'][trading_str_b['trade']==1].count()  
b = trading_str_b['BTC_dir'][(trading_str_b['trade']==1) & (trading_str_b['BTC_dir']==1)].count()

c = trading_str_b['BTC_dir'][trading_str_b['trade']==-1].count()
d = trading_str_b['BTC_dir'][(trading_str_b['trade']==-1) & (trading_str_b['BTC_dir']==0)].count()

# Printing out accuracy of predictions
print('Predictions Window-size:', len(trading_str_b), '\n', 'Total Predictions Made:', a+c, '\n', 'Correct Predictions:', b+d)
print('Overall Accuracy:', round((b+d)/(a+c)*100,1),'%')

print('UP Predictions Made:', a, '\n', 'Correct UP Predictions:', b, '\n', 'DOWN Predictions Made:', c, '\n', 'Correct DOWN Predictions:', d)
print('UP Accuracy:', (b)/(a)*100,'%') 
print('DOWN Accuracy:', (d)/(c)*100,'%')

Predictions Window-size: 132 
 Total Predictions Made: 44 
 Correct Predictions: 31
Overall Accuracy: 70.5 %
UP Predictions Made: 30 
 Correct UP Predictions: 19 
 DOWN Predictions Made: 14 
 Correct DOWN Predictions: 12
UP Accuracy: 63.33333333333333 %
DOWN Accuracy: 85.71428571428571 %


In [42]:
trading_str_b = trading_str_b[["Date", "BTC_dir", "trade"]]
bit_df = bit_2[["Date", "BTC_Change%"]]
trading_str_b = trading_str_b.merge(bit_df, on="Date", how="left" )
# Dropping first row
# trading_str_b = trading_str_b.iloc[1: , :]
trading_str_b["return_bit"] = trading_str_b["trade"] * trading_str_b["BTC_Change%"] 
trading_str_b["total_ret_b"] = trading_str_b["return_bit"] + 1
trading_str_b["cum_tot_ret_b"] = trading_str_b["total_ret_b"].cumprod()
# We are assuming that we are starting from investment of USD 10,000.
trading_str_b["bit_inv_value"] = round(trading_str_b["cum_tot_ret_b"] * 10000 ,2)

# Trading with STOP LOSS at 5%
sl_level = 0.05
trading_str_b["total_ret_b_sl"] = np.where(trading_str_b["return_bit"]<-sl_level, -sl_level, trading_str_b["return_bit"]) + 1
trading_str_b["cum_tot_ret_b_sl"] = trading_str_b["total_ret_b_sl"].cumprod()
# We are assuming that we are starting from investment of USD 10,000.
trading_str_b["bit_inv_value_sl"] = round(trading_str_b["cum_tot_ret_b_sl"] * 10000 ,2)


trading_str_b['month'] = trading_str_b['Date'].dt.month

ls = [4, 5, 6, 7, 8, 9, 10]
for i in ls:
    trading_str_b.loc[trading_str_b['month']==i, 'max_date'] = trading_str_b['Date'][trading_str_b['month']==i].max()
     
trading_str_b.loc[trading_str_b["Date"] == trading_str_b["max_date"], "ret_value_b"] = trading_str_b["bit_inv_value"] - 10000
trading_str_b.loc[trading_str_b["Date"] == trading_str_b["max_date"], "ret_value_b_sl"] = trading_str_b["bit_inv_value_sl"] - 10000

trading_str_b = trading_str_b.drop(columns=['month', 'max_date'])   


trading_str_b = trading_str_b[["Date", "BTC_dir", "trade", "bit_inv_value", "bit_inv_value_sl", "ret_value_b", "ret_value_b_sl"]]
trading_str_b


,Date,BTC_dir,trade,bit_inv_value,bit_inv_value_sl,ret_value_b,ret_value_b_sl
0,2021-03-31,1.0,0,10000.00,10000.00,NaN,NaN
1,2021-04-01,1.0,0,10000.00,10000.00,NaN,NaN
2,2021-04-05,0.0,0,10000.00,10000.00,NaN,NaN
3,2021-04-06,0.0,1,9853.45,9853.45,NaN,NaN
4,2021-04-07,0.0,0,9853.45,9853.45,NaN,NaN
...,...,...,...,...,...,...,...
127,2021-09-30,1.0,1,21237.67,23236.79,11237.67,13236.79
128,2021-10-01,1.0,0,21237.67,23236.79,NaN,NaN
129,2021-10-04,1.0,0,21237.67,23236.79,NaN,NaN
130,2021-10-05,1.0,0,21237.67,23236.79,NaN,NaN


In [43]:
trading_str_b.to_csv("trading_str_b.csv", index=False)

In [44]:
filename = 'bit_rf.sav'
joblib.dump(random_forest, filename)

['bit_rf.sav']

In [45]:
filename = 'bit_XG.sav'
joblib.dump(xg_m, filename)

['bit_XG.sav']

In [46]:
filename = 'bit_XGR.sav'
joblib.dump(xg_mr, filename)

['bit_XGR.sav']

In [47]:
filename = 'bit_logit.sav'
joblib.dump(logit, filename)

['bit_logit.sav']